<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/AV_SVM_EVAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
!pip install -U transformers
!pip install -U accelerate

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
import torch
import re
import zipfile

# Load dev data set

In [ ]:
dev_corpus = pd.read_csv("dev.csv", encoding='utf-8')
dev_labels = np.array(dev_corpus['label'])
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# Load Model

In [ ]:
with zipfile.ZipFile('SVM_MODEL.zip', 'r') as zip_ref:
    zip_ref.extractall('SVM_MODEL')

SVM_MODEL = tf.keras.models.load_model("SVM_MODEL/content/AV_SVM_MODEL")

#Prepare data

In [ ]:
def preprocess(string):
  output = str(string).lower()
  separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

# Prepare data for Distilled Bert
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(dev_corpus)


SEQ_LENGTH = 128
BATCH_SIZE = 32

def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)

# Test Models

In [ ]:
predictions = SVM_MODEL.predict(input_data)
binary_predictions = (predictions >= 0.5).astype(int)

#Save predictions

In [ ]:
predictions_DF = pd.DataFrame(binary_predictions, columns=['prediction'])
predictions_DF.to_csv('dev_predictions.csv', index=False)